
<a href="https://githubtocolab.com/Astolfo2332/VIU_act/blob/main/optimizacion/Algoritmos_TP.ipynb" target="_parent\">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" > </a>


# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Miguel López Vélez <br>
Url: [Trabajo Práctico](https://github.com/Astolfo2332/VIU_act/blob/main/optimizacion/Algoritmos_TP.ipynb)<br>
Problema:
> 1. Sesiones de doblaje <br>
> 2. Organizar los horarios de partidos de La Liga<br>
> 3. Combinar cifras y operaciones

Descripción del problema:

- El problema consiste en analizar el siguiente problema y diseñar un algoritmo que lo resuelva.
- Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/)
- Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una cantidad dada. Un ejemplo sería para obtener el 4:

$$
4+2-6/3*1 = 4
$$

Debe analizarse el problema para encontrar todos los valores enteros posibles planteando las siguientes cuestiones:
- ¿Qué valor máximo y mínimo se pueden obtener según las condiciones del problema?
- ¿Es posible encontrar todos los valores enteros posibles entre dicho mínimo y máximo?



(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

Inicialmente, debemos determinar las restricciones del problema, las cuales son las siguientes:
- No se pueden repetir los números
- No se pueden repetir los signos
- Se deben alternar los números y los signos
- Los resultados deben ser enteros

De esta forma la cantidad de posibilidades se limitan por el número de signos, ya que solo podemos usar 4 a la vez limita el uso de los números a 5, y estos al no poder repetirse lo vuelve un problema de permutación, porque el orden no importa:

$$
P_{9,5} = \frac{9!}{9!-5!} = 15120
$$

A esto se le debe multiplicar la cantidad de posibles combinaciones de signos sin repetir, lo cual es un problema de permutaciones de la forma:

$$
P_{4} = 4! = 24
$$

Dando como resultado un espacio de:

$$
15120 * 24 = 362880 posibilidades
$$

Ahora agregando la restricción de que el resultado debe ser un número entero, se debe analizar el rango de valores que se pueden obtener y evaluar si pertenecen o no al conjunto de enteros.


Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, argumentalo)

Se puede implementar un diccionario para almacenar el valor de los resultados en la clave y que su valor sea una lista de expresiones que den como resultado dicho valor, facilitando la búsqueda de los valores enteros.

Respuesta

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?
Estableciendo
$$
N = \{1, 2, 3, 4, 5, 6, 7, 8, 9\}
$$

y

$$
O = \{+, -, *, /\}
$$

$$
f(N, O) = \{x \in \mathbb{Z} \ | \ \text{existe una combinación válida de } N \text{ y } O \text{ tal que } E(N, O) = x\}
$$
(*)¿Es un problema de maximización o minimización?

Ambos ya que debemos establecer el máximo y mínimo de la expresión dada.

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [7]:
from itertools import permutations

res_dict = {}

def eval_expresssion(exp, res_dict):
    result = eval(exp)
    if result.is_integer():
        if result in res_dict:
            res_dict[result].append(exp)
        else:
            res_dict[result] = [exp]
    return res_dict

def get_expressions(res_dict):
    #Generamos las permutaciones de numeros
    for nums in permutations('123456789', 5):
        #Generamos las permutaciones de operaciones
        for ops in permutations('+-*/', 4):
            #Unimos los números y las operaciones, recordemos que se pone el número adicional al final
            expression = ''.join([f'{nums[i]}{ops[i]}' for i in range(4)]) + nums[4]
            #Se evalua el valor de la expresión
            res_dict = eval_expresssion(expression, res_dict)
    return res_dict

res_dict = get_expressions(res_dict)


In [56]:
def continuidad(res_dict):
    min_res = min(res_dict.keys())
    max_res = max(res_dict.keys())
    for i in range(int(min_res), int(max_res)+1):
        if i not in res_dict:
            return False, min_res, max_res
    return True, min_res, max_res

In [57]:
continuo, min_res, max_res = continuidad(res_dict)

print("Mínimo:", min_res)
print("Máximo:", max_res)
print("Es continuo el resultado?", "si" if continuo else "no")

Mínimo: -69.0
Máximo: 77.0
Es continuo el resultado? si


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

En este caso la complejidad es dada por el numero de permutaciones de los números y las operaciones, si lo definimos como $m$ y $n$ respectivamente, la complejidad es de $O(m * n)$

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

En este caso podemos mejorar la complejidad del algoritmo mediante el uso de un algoritmo de backtracking, ya que podemos ir evaluando las expresiones y si no cumplen con la condición de ser un número entero, no es necesario seguir evaluando las demás expresiones.

También se puede implementar una función de programación dinámica para almacenar los resultados de las expresiones y no tener que evaluarlas nuevamente.

Y lo más importante se debe maximizar y minimizar el rango de valores, así que podemos tomar desde establecer que la operación de multiplicación y división sea un entero buscar en ese rango cuál es el valor más grande y más pequeño que obtener a partir de las combinaciones de los números restantes.

De esta forma podríamos establecer un registro de operaciones fijo, ya que si se observa solo las iteraciones de números se puede lograr tanto el mismo resultado



In [52]:
class Solution:
    def __init__(self, problem, operations):
        self.problem = problem
        self.operations = ["/", "*", "+", "-"]
        self.rest_dict = {}
        self.cache_data = set()
        self.result = []

    def validate_data(self, exp, is_max:bool=True):
        tuple_exp = tuple(exp)
        exp = list(exp)
        exp = [str(a) for a in exp]
        if is_max:
            exp = exp[0] + "/" + exp[1] + "*" + exp[2] + "+" + exp[3] + "-" + exp[4]
        else:
            exp =  exp[3] + "+" + exp[4] + "-" + exp[0] + "/" + exp[1] + "*" + exp[2]
        result = eval(exp)

        if result not in self.rest_dict:
            self.rest_dict[result] = [tuple_exp]
        else:
            self.rest_dict[result].append(tuple_exp)
        return result

    def is_integer(self, data):
        data = [int(a) for a in data]
        if data[0] % data[1] == 0:
            return True
        elif data[2] % data[1] == 0:
            return True
        return False

    def backtrack(self, path:list, used:tuple):
        if len(path) == 3:
            if self.is_integer(path):
                self.result.append(tuple(path))

                max_path = self.add_and_sub(path, True)
                min_path = self.add_and_sub(path, False)

                self.validate_data(max_path)
                self.validate_data(min_path)

                self.validate_data(max_path, False)
                self.validate_data(min_path, False)

                return

        for num in self.problem:
            num = int(num)
            if num not in used:
                used.add(num)
                path.append(num)
                self.backtrack(path, used)
                #Se reinicia si no se hace el stop
                path.pop()
                used.remove(num)

    def add_and_sub(self, path:list, is_max:bool):

        problem = [int(a) for a in self.problem]
        used = set(path)
        not_used = set(problem) - used

        max_val = max(not_used)
        min_val = min(not_used)

        new_path = path.copy()

        if is_max:
            new_path.append(max_val)
            new_path.append(min_val)
        else:
            new_path.append(min_val)
            new_path.append(max_val)

        return new_path




    def find_values(self):
        self.backtrack([], set())

        return self.result



a = '123456789'


sol = Solution(a, "")

a = sol.find_values()







In [54]:
min(sol.rest_dict.keys()), max(sol.rest_dict.keys())

(-63.0, 77.0)

['1+4/2-8*9',
 '1+4/2-9*8',
 '1+6/3-8*9',
 '1+6/3-9*8',
 '1-8*9+4/2',
 '1-8*9+6/3',
 '1-9*8+4/2',
 '1-9*8+6/3',
 '4/2+1-8*9',
 '4/2+1-9*8',
 '4/2-8*9+1',
 '4/2-9*8+1',
 '6/3+1-8*9',
 '6/3+1-9*8',
 '6/3-8*9+1',
 '6/3-9*8+1']

In [42]:
import re

class Solution:
    def __init__(self, problem, operations):
        self.problem = problem
        self.operations = operations
        self.rest_dict = {}
        self.cache_data = set()
        self.result = []
        self.pattern = r"\d+/\d+"
        self.possible_mul = []
        self.path = []
        self.used = set()
        self.used_op = []

    def backtrack(self):

        sanity = self.sanity_check()
        if not sanity:
            return

        if len(self.path) == 9:
            path = "".join(self.path)
            result = eval(path)
            print(path, result)
            if result.is_integer() and path not in self.result:
                self.result.append(path)
                if result not in self.rest_dict:
                    self.rest_dict[result] = [path]
                else:
                    self.rest_dict[result].append(path)

                return


        if len(self.path) % 2 == 0:
            for num in self.problem:
                if num in self.used:
                    continue
                self.path.append(num)
                self.used.add(num)
                self.backtrack()
                self.path.pop()
                self.used.remove(num)
        else:
            for op in self.operations:
                if op in self.used_op:
                    continue
                self.path.append(op)
                self.used_op.append(op)
                self.backtrack()
                self.path.pop()
                self.used_op.pop()


    def sanity_check(self):
        #Check para ver si es un entero y posee la división
        if len(self.path) == 0:
            return True

        if self.path[-1] in self.problem and "/" in self.path:
            if not eval(''.join(self.path)).is_integer() and "*" in self.path and len(self.path) == 9:
                return False

            div = re.findall(self.pattern, ''.join(self.path))[0]
            div = div.split("/")[1]
            not_used = set(self.problem) - self.used
            #Evaluamos los multiplos que vuelvan entero el resultado
            poss_mult = [a for a in not_used if int(a) % int(div) == 0]

            if len(poss_mult) == 0:
                return False

            #Procesamos los multiplos para obtener el resultado que no este guardado
            poss_mult = [str(a) for a in poss_mult]
            join_path = ''.join(self.path)
            poss_mult = [join_path + "*" + a for a in poss_mult]
            poss_mult = [a for a in poss_mult if a not in self.result]
            poss_mult = poss_mult[0].split("*")[1]

            # #Modificamos el path con los nuevos operadores
            # self.path.extend(["*", poss_mult])
            # self.used.add(poss_mult)
            # self.used_op.append("*")

            return True
        return True

    def find_values(self):
        self.backtrack()
        return self.result





a = '123456789'
operations = ['+', '-', '*', '/']


sol = Solution(a, operations)

a = sol.find_values()


1+2-4*6/3 -5.0
1+2-4*9/3 -9.0
1+2-4/3*6 -5.0
1+2-4/3*9 -9.0
1+2-5*6/3 -7.0
1+2-5*9/3 -12.0
1+2-5/3*6 -7.0
1+2-5/3*9 -12.0
1+2-6*4/3 -5.0
1+2-6*5/3 -7.0
1+2-6*7/3 -11.0
1+2-6*8/3 -13.0
1+2-6/3*4 -5.0
1+2-6/3*5 -7.0
1+2-6/3*7 -11.0
1+2-6/3*8 -13.0
1+2-7*6/3 -11.0
1+2-7*9/3 -18.0
1+2-7/3*6 -11.0
1+2-7/3*9 -18.0
1+2-8*6/3 -13.0
1+2-8*9/3 -21.0
1+2-8/3*6 -13.0
1+2-8/3*9 -21.0
1+2-9*4/3 -9.0
1+2-9*5/3 -12.0
1+2-9*7/3 -18.0
1+2-9*8/3 -21.0
1+2-9/3*4 -9.0
1+2-9/3*5 -12.0
1+2-9/3*7 -18.0
1+2-9/3*8 -21.0
1+2*4-6/3 7.0
1+2*4-9/3 6.0
1+2*5-6/3 9.0
1+2*5-9/3 8.0
1+2*6/3-4 1.0
1+2*6/3-5 0.0
1+2*6/3-7 -2.0
1+2*6/3-8 -3.0
1+2*6/4-3 1.0
1+2*6/4-5 -1.0
1+2*6/4-7 -3.0
1+2*6/4-9 -5.0
1+2*7-6/3 13.0
1+2*7-9/3 12.0
1+2*8-6/3 15.0
1+2*8-9/3 14.0
1+2*9/3-4 3.0
1+2*9/3-5 2.0
1+2*9/3-7 0.0
1+2*9/3-8 -1.0
1+2/3*6-4 1.0
1+2/3*6-5 0.0
1+2/3*6-7 -2.0
1+2/3*6-8 -3.0
1+2/3*9-4 3.0
1+2/3*9-5 2.0
1+2/3*9-7 0.0
1+2/3*9-8 -1.0
1+2/4*6-3 1.0
1+2/4*6-5 -1.0
1+2/4*6-7 -3.0
1+2/4*6-9 -5.0
1+3-2*6/4 1.0
1+3-2/4*6 1.0
1+3-4*5/

(*)Calcula la complejidad del algoritmo



Respuesta

In [43]:
print(max(sol.rest_dict.keys()), min(sol.rest_dict.keys()))

77.0 -69.0


Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta